In [31]:
import sys
import numpy as np
import cv2
import sklearn.metrics as sklm
import tensorflow as tf

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model, load_model

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,Input
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential

from tensorflow.keras import backend as K
from tensorflow.keras import regularizers

from keras import backend as K

In [32]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("dimensions before ",X_train.shape)
# print(y_train.shape)

dim = (32, 32)

#convert 28x28 grayscale to 48x48 rgb channels
def to_rgb(img):
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
    img_rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    return img_rgb

rgb_list = []
#convert X_train data to 48x48 rgb values
for i in range(len(X_train)):
    rgb = to_rgb(X_train[i])
    rgb_list.append(rgb)

    
rgb_arr = np.stack([rgb_list],axis=4)
rgb_arr_to_3d = np.squeeze(rgb_arr, axis=4)

print("dimensions after",rgb_arr_to_3d.shape)

dimensions before  (60000, 28, 28)
dimensions after (60000, 32, 32, 3)


In [33]:
#another 3d conversion method that failed

# (X_train, y_train), (X_test, y_test) = mnist.load_data()

# X_train = X_train / 255.
# X_test = X_test / 255.


# X_train=tf.image.grayscale_to_rgb(tf.expand_dims(X_train, axis=3)) 
# X_test=tf.image.grayscale_to_rgb(tf.expand_dims(X_test, axis=3))
# # print(X_train.shape)
# # print(y_train.shape)

# X_train=tf.image.resize_with_pad(X_train,32 ,32)
# X_test=tf.image.resize_with_pad(X_test,32,32)
# # print(X_train.shape)
# # print(y_train.shape)


# #preprocessing input
# X_train=preprocess_input(X_train)
# X_test=preprocess_input(X_test)

# X_train = X_train / 255.
# X_test = X_test / 255.
# print(X_train.shape)
# print(y_train.shape)

In [34]:
# vff_model = VGG16()
vff_model=tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(32,32,3))


In [35]:
vff_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [36]:
model = Sequential()
for layer in vff_model.layers[:-1]:
    model.add(layer)

In [37]:
for layer in model.layers:
    layer.trainable == False
model.add(Flatten())
model.add(Dense(units=10,activation='softmax'))

In [38]:
model.summary()
print(X_train.shape)
print(X_test.shape)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 8, 8, 256)        

In [39]:
model.compile(optimizer = 'adam' , loss='categorical_crossentropy' , metrics = ['accuracy'] )
y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)
print(y_train.shape)
print(y_test.shape)

(60000, 10)
(10000, 10)


In [42]:
model.fit(rgb_arr_to_3d,y_train,epochs=5,verbose=1,batch_size=128)

Epoch 1/5
469/469 [==============================] - 25s 52ms/step - loss: 0.0423 - accuracy: 0.9888
Epoch 2/5
469/469 [==============================] - 24s 52ms/step - loss: 0.0284 - accuracy: 0.9923
Epoch 3/5
469/469 [==============================] - 24s 51ms/step - loss: 0.0200 - accuracy: 0.9941
Epoch 4/5
469/469 [==============================] - 24s 51ms/step - loss: 0.0142 - accuracy: 0.9958
Epoch 5/5
469/469 [==============================] - 24s 51ms/step - loss: 0.0113 - accuracy: 0.9969


In [43]:
dim = (32, 32)

#convert 28x28 grayscale to 48x48 rgb channels
def to_rgb(img):
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
    img_rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    return img_rgb

t_rgb_list = []
#convert X_train data to 48x48 rgb values
for i in range(len(X_test)):
    rgb = to_rgb(X_test[i])
    t_rgb_list.append(rgb)

    
rgb_arr = np.stack([t_rgb_list],axis=4)
t_rgb_arr_to_3d = np.squeeze(rgb_arr, axis=4)

model.evaluate(t_rgb_arr_to_3d,y_test)

313/313 [==============================] - 3s 9ms/step - loss: 0.0379 - accuracy: 0.9912


[0.037866853177547455, 0.9911999702453613]